In [14]:
pip install tabpfn-extensions

  Using cached seaborn-0.12.2-py3-none-any.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 766.5 kB/s eta 0:00:00a 0:00:01
Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.13.2
    Uninstalling seaborn-0.13.2:
      Successfully uninstalled seaborn-0.13.2

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas  as pd
from sklearn.datasets import load_diabetes
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tabpfn import TabPFNRegressor
from sklearn.discriminant_analysis import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
import os
from utils import run_kfold_eval, calculate_metric
import optuna
import time
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import KFold
from tabpfn_extensions.interpretability import get_shap_values, plot_shap
from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNRegressor

In [2]:
DATA_FOLDER = "../data"
TRAIN_FEATURES = DATA_FOLDER + '/train_features2.xlsx'
TRAIN_LABELS = DATA_FOLDER + "/train_labels2.xlsx"
TEST_FEATURES = DATA_FOLDER + "/test_features2.xlsx"
TEST_LABELS = DATA_FOLDER + "/test_labels2.xlsx"
OUTPUT_PATH = '../output/tabpfn'

TRAINING_OUTPUT_FILE = '../output/train_predictions.xlsx'
TEST_OUTPUT_FILE = '../output/test_predictions.xlsx'
SHEET_NAME = "tabpfn"

In [3]:
train_features = pd.read_excel(TRAIN_FEATURES)
train_labels = pd.read_excel(TRAIN_LABELS)
test_features = pd.read_excel(TEST_FEATURES)
test_labels = pd.read_excel(TEST_LABELS)

In [4]:
test_features.head()

,coupon rate,SP500 MD,Average daily 1-year SP500 return,Ratio to MA,US Corporate Bond Yield Spread,US Corporate Bond Yield Spread(3-5 year),US Corporate Bond Yield Spread(5-7 year),US Corporate Bond Yield Spread(7-10 year),US Corporate Bond Yield Spread(10+ year),US Generic Govt 3 Month Yield,...,event_type_subcategory_sum_Missing Coupon payment only,event_type_subcategory_sum_Missing Interest payment,event_type_subcategory_sum_Missing Loan payment,event_type_subcategory_sum_Missing Principal payment,event_type_subcategory_sum_Others,event_type_subcategory_sum_Pre-Negotiated Chapter 11,event_type_subcategory_sum_Protection,event_type_subcategory_sum_Receivership,event_type_subcategory_sum_Rehabilitation,event_type_subcategory_sum_Restructuring
0,3.125,-122.39680,-0.001341,1.116103,396.408936,399.031738,455.0625,383.612488,341.675812,0.1734,...,False,False,False,False,False,False,False,False,False,False
1,15.000,156.01912,0.000790,6.372588,107.083778,76.414360,118.2482,113.403519,147.756165,0.0253,...,False,False,False,False,False,False,False,False,False,False
2,0.000,-451.10824,-0.002033,43.914617,637.607544,689.770996,683.3611,589.969666,529.166992,0.0406,...,False,False,False,True,False,False,False,False,False,False
3,5.875,-751.22120,-0.000435,74.783398,153.653778,129.425842,158.3252,172.708054,187.147980,1.4908,...,False,False,False,False,False,False,False,False,False,False
4,10.000,55.69036,0.000712,-28.522133,152.290619,148.385330,222.1811,142.140915,159.624313,0.0763,...,False,False,False,False,False,False,False,False,False,False


In [5]:
print(train_features.shape)
print(train_labels.shape)
print(test_features.shape)
print(test_labels.shape)

(1293, 317)
(1293, 1)
(432, 317)
(432, 1)


In [6]:
feature_list = train_features.columns
# category_feature_key = ['currency', 'seniorioty_adj', 'domicile_country',	'exchange_country',	'Industry_sector',	'Industry_group',	'Industry_subgroup', 'event_type',
#                  'event_type_subcategory_sum', 'defaulted_in_last_6_months']
# category_features = [i for i in feature_list if any(sub in i for sub in category_feature_key)]
category_features = train_features.select_dtypes(include=['float']).columns
non_category_features = [i for i in feature_list if i not in category_features]

print(len(category_features))
print(len(non_category_features))

153
164


In [7]:
non_category_features

['defaulted_in_last_5_years',
 'defaulted_in_last_6_months',
 'currency_CAD',
 'currency_CHF',
 'currency_CNY',
 'currency_EUR',
 'currency_GBP',
 'currency_HKD',
 'currency_INR',
 'currency_ISK',
 'currency_JPY',
 'currency_MYR',
 'currency_NOK',
 'currency_SEK',
 'currency_SGD',
 'currency_THB',
 'currency_TWD',
 'currency_USD',
 'seniorioty_adj_Junior Unsecured or Junior Subordinated Unsecured',
 'seniorioty_adj_Secured',
 'seniorioty_adj_Senior Secured',
 'seniorioty_adj_Senior Subordinated Unsecured',
 'seniorioty_adj_Senior Unsecured',
 'seniorioty_adj_Subordinated Unsecured',
 'seniorioty_adj_Unsecured',
 'domicile_country_Argentina',
 'domicile_country_Australia',
 'domicile_country_Bahamas',
 'domicile_country_Belgium',
 'domicile_country_Bermuda',
 'domicile_country_Canada',
 'domicile_country_Cayman Islands',
 'domicile_country_China',
 'domicile_country_Czech Republic',
 'domicile_country_Greece',
 'domicile_country_Hong Kong',
 'domicile_country_Iceland',
 'domicile_countr

In [8]:
# Normalize the data
# Prepare the ColumnTransformer

scaler = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), non_category_features)   # StandardScaler()
    ],
    remainder='passthrough'  # Leave categorical features untouched
)

In [9]:
train_features_scaled = scaler.fit_transform(train_features)
test_features_scaled = scaler.transform(test_features)

In [10]:
reg = TabPFNRegressor()
reg.fit(train_features_scaled, train_labels)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


TabPFNRegressor()

In [ ]:
# Predict a point estimate (using the mean)
predictions = reg.predict(test_features_scaled)
print("Mean Squared Error (MSE):", mean_squared_error(test_labels, predictions))
print("Mean Absolute Error (MAE):", mean_absolute_error(test_labels, predictions))
print("R-squared (R^2):", r2_score(test_labels, predictions))

In [48]:
# save metrics
mae, mape, rmse, rsqr = calculate_metric(predictions, test_labels['rr1_30'].to_numpy())
print(f"Val average mean absolute error: {mae}")
print(f"Val average mean absolute percentage error: {mape}")
print(f"Val average root mean squared error: {rmse}")
print(f"Val average R2: {rsqr}")

Val average mean absolute error: 0.15547700572670708
Val average mean absolute percentage error: 295.0666177615481
Val average root mean squared error: 0.21969386545389463
Val average R2: 0.578630067729488


In [ ]:
train_predictions = reg.predict(train_features_scaled)
# save metrics
mae, mape, rmse, rsqr = calculate_metric(train_predictions, train_labels['rr1_30'].to_numpy())
print(f"Val average mean absolute error: {mae}")
print(f"Val average mean absolute percentage error: {mape}")
print(f"Val average root mean squared error: {rmse}")
print(f"Val average R2: {rsqr}")

In [ ]:
# save train predictions
train_predictions_df = pd.DataFrame({"predictions": train_predictions})
with pd.ExcelWriter(TRAINING_OUTPUT_FILE, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    # Write the new DataFrame to a new sheet
    train_predictions_df.to_excel(writer, sheet_name=SHEET_NAME, index=False)
    
# save test predictions
test_predictions_df = pd.DataFrame({"predictions": predictions})
with pd.ExcelWriter(TEST_OUTPUT_FILE, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    # Write the new DataFrame to a new sheet
    test_predictions_df.to_excel(writer, sheet_name=SHEET_NAME, index=False)

SHAP

In [ ]:
# Compute SHAP values
shap_values = get_shap_values(reg, test_features_scaled)

# Visualize feature importance
plot_shap(shap_values)

In [ ]:
# plt.figure(figsize=(12, 8))
# shap.summary_plot(kernel_shap_values, X_test_sample, show=False)
# plt.title('TabPFN Feature Impact (SHAP values)')
# plt.tight_layout()
# plt.savefig(f'{OUTPUT_PATH}/tabpfn_shap_summary.png')
# plt.close()

Ensemble

In [13]:
reg_en = AutoTabPFNRegressor(max_time=120) # 120 seconds tuning time
reg_en.fit(train_features_scaled, train_labels)
predictions = reg_en.predict(test_features_scaled)

2025-02-05 16:01:52 INFO     Using `default` preset for Post Hoc Ensemble.
2025-02-05 16:01:52 INFO     No categorical_feature_indices given. Assuming no categorical features.
/Users/hoanguyen/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Recovery rate forecasting/Code/sparseGP_powerEP/python/sgp/venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/hoanguyen/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Recovery rate forecasting/Code/sparseGP_powerEP/python/sgp/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/hoanguyen/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Recovery rate forecasting/Cod

In [14]:
# save metrics
mae, mape, rmse, rsqr = calculate_metric(predictions, test_labels['rr1_30'].to_numpy())
print(f"Val average mean absolute error: {mae}")
print(f"Val average mean absolute percentage error: {mape}")
print(f"Val average root mean squared error: {rmse}")
print(f"Val average R2: {rsqr}")

Val average mean absolute error: 0.15333190566201182
Val average mean absolute percentage error: 309.6688697840275
Val average root mean squared error: 0.21848332496447248
Val average R2: 0.5832608756596755


5-fold CV

In [16]:
features = pd.concat([train_features, test_features], axis=0, ignore_index=True)
features

,coupon rate,SP500 MD,Average daily 1-year SP500 return,Ratio to MA,US Corporate Bond Yield Spread,US Corporate Bond Yield Spread(3-5 year),US Corporate Bond Yield Spread(5-7 year),US Corporate Bond Yield Spread(7-10 year),US Corporate Bond Yield Spread(10+ year),US Generic Govt 3 Month Yield,...,event_type_subcategory_sum_Missing Coupon payment only,event_type_subcategory_sum_Missing Interest payment,event_type_subcategory_sum_Missing Loan payment,event_type_subcategory_sum_Missing Principal payment,event_type_subcategory_sum_Others,event_type_subcategory_sum_Pre-Negotiated Chapter 11,event_type_subcategory_sum_Protection,event_type_subcategory_sum_Receivership,event_type_subcategory_sum_Rehabilitation,event_type_subcategory_sum_Restructuring
0,7.500,-117.46020,-0.000189,125.407139,177.213028,134.012054,198.8153,191.364395,223.346344,0.1983,...,True,False,False,False,False,False,False,False,False,False
1,6.000,166.38276,0.000768,-4.603446,101.613617,77.032829,123.3998,105.932022,139.111115,0.0355,...,False,False,False,False,False,False,False,False,False,False
2,11.000,119.85752,0.000678,-11.950380,104.545959,77.416649,129.4317,111.818001,139.717407,0.0101,...,False,False,False,False,False,False,False,False,False,False
3,9.125,653.51208,0.001638,-2.494861,90.736633,64.654129,95.3731,92.141212,121.666237,0.0152,...,False,False,False,False,False,False,False,False,False,False
4,9.250,231.89472,0.000664,4.823413,98.533821,68.759308,93.4174,107.424469,139.741165,1.2865,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1720,3.250,147.33344,0.000556,3.538252,128.976395,95.360374,138.8445,138.946106,172.733887,0.0000,...,False,False,False,False,False,False,False,False,False,False
1721,10.000,175.31656,0.000554,-4.067337,121.170998,92.879501,123.2500,131.104904,162.916901,0.7516,...,False,True,False,False,False,False,False,False,False,False
1722,7.450,315.81748,0.000747,8.604100,126.786606,89.018188,114.9728,131.522430,171.701096,0.0864,...,False,False,False,False,False,False,False,False,False,False
1723,0.500,31.75120,-0.000157,-4.373852,126.595230,105.460007,121.6212,147.382416,163.772141,4.8375,...,False,False,False,False,False,False,False,False,False,False


In [17]:
labels = pd.concat([train_labels, test_labels], axis=0, ignore_index=True)
labels

,rr1_30
0,0.082481
1,0.378845
2,0.836149
3,0.987208
4,1.021458
...,...
1720,0.471411
1721,0.823750
1722,0.241612
1723,0.762054


In [20]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import numpy as np
from tqdm import tqdm


In [ ]:
# Define the function to run k-fold cross-validation

# Initialize dictionaries to store metrics
train_metrics = {}
val_metrics = {}
feat_importances = {}

# Create KFold object with random seed 42
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [41]:


# Loop through folds
for fold, (train_idx, val_idx) in enumerate(tqdm(kf.split(features))):
    if fold < 2:
        continue
    print(f"fold {fold}")
    # Split data
    X_train, X_val = features.iloc[train_idx], features.iloc[val_idx]
    y_train, y_val = labels.iloc[train_idx], labels.iloc[val_idx]
    
    # Initialize and fit TabPFN model
    tabpfn_model = TabPFNRegressor()
    
    # Scale features
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    # Fit model
    tabpfn_model.fit(X_train_scaled, y_train)
    
    # Make predictions
    train_fold_preds = tabpfn_model.predict(X_train_scaled)
    val_fold_preds = tabpfn_model.predict(X_val_scaled)
    

    # Calculate metrics
    train_mae, train_mape, train_rmse, train_rsqr = calculate_metric(train_fold_preds, y_train.values.ravel())
    val_mae, val_mape, val_rmse, val_rsqr = calculate_metric(val_fold_preds, y_val.values.ravel())
    
    # Store metrics
    train_metrics[f'fold_{fold}_mae'] = train_mae
    train_metrics[f'fold_{fold}_mape'] = train_mape
    train_metrics[f'fold_{fold}_rmse'] = train_rmse
    train_metrics[f'fold_{fold}_rsqr'] = train_rsqr
    
    val_metrics[f'fold_{fold}_mae'] = val_mae
    val_metrics[f'fold_{fold}_mape'] = val_mape
    val_metrics[f'fold_{fold}_rmse'] = val_rmse
    val_metrics[f'fold_{fold}_rsqr'] = val_rsqr
    
    # Print fold results
    print(f"Fold {fold+1} - Train MAE: {train_mae:.4f}, RMSE: {train_rmse:.4f}, R²: {train_rsqr:.4f}")
    print(f"Fold {fold+1} - Val MAE: {val_mae:.4f}, RMSE: {val_rmse:.4f}, R²: {val_rsqr:.4f}")



0it [00:00, ?it/s]/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


fold 2


3it [5:32:36, 6652.28s/it]

Fold 3 - Train MAE: 0.0799, RMSE: 0.1245, R²: 0.8585
Fold 3 - Val MAE: 0.1548, RMSE: 0.2327, R²: 0.5170
fold 3


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
4it [7:23:50, 6659.06s/it]

Fold 4 - Train MAE: 0.0982, RMSE: 0.1472, R²: 0.8071
Fold 4 - Val MAE: 0.1461, RMSE: 0.2080, R²: 0.5706
fold 4


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
5it [9:15:30, 6666.18s/it]

Fold 5 - Train MAE: 0.0851, RMSE: 0.1321, R²: 0.8363
Fold 5 - Val MAE: 0.1483, RMSE: 0.2138, R²: 0.6294


In [44]:
val_metrics

{'fold_0_mae': 0.1451019096742838,
 'fold_0_mape': 64.21223903389837,
 'fold_0_rmse': 0.2098485538683658,
 'fold_0_rsqr': 0.5895800175220378,
 'fold_1_mae': 0.14169854683111766,
 'fold_1_mape': 76.53111264040359,
 'fold_1_rmse': 0.21957764170941382,
 'fold_1_rsqr': 0.5445326356375579,
 'fold_2_mae': 0.15479387928882873,
 'fold_2_mape': 22875.271080225957,
 'fold_2_rmse': 0.23270570543796848,
 'fold_2_rsqr': 0.5170220465302433,
 'fold_3_mae': 0.14613352282329375,
 'fold_3_mape': 85.9666865733556,
 'fold_3_rmse': 0.20796374325078848,
 'fold_3_rsqr': 0.5705990521440356,
 'fold_4_mae': 0.14829968217274386,
 'fold_4_mape': 93.12177989841626,
 'fold_4_rmse': 0.2137654952210911,
 'fold_4_rsqr': 0.6293517596053302}

In [43]:

# Calculate and print average metrics
print("\nTabPFN 5-fold CV Results:")
print(f"Average Train MAE: {np.mean([v for k, v in train_metrics.items() if 'mae' in k]):.4f}")
print(f"Average Train RMSE: {np.mean([v for k, v in train_metrics.items() if 'rmse' in k]):.4f}")
print(f"Average Train R²: {np.mean([v for k, v in train_metrics.items() if 'rsqr' in k]):.4f}")
print(f"Average Val MAE: {np.mean([v for k, v in val_metrics.items() if 'mae' in k]):.4f}")
print(f"Average Val RMSE: {np.mean([v for k, v in val_metrics.items() if 'rmse' in k]):.4f}")
print(f"Average Val R²: {np.mean([v for k, v in val_metrics.items() if 'rsqr' in k]):.4f}")



TabPFN 5-fold CV Results:
Average Train MAE: 0.0872
Average Train RMSE: 0.1340
Average Train R²: 0.8361
Average Val MAE: 0.1472
Average Val RMSE: 0.2168
Average Val R²: 0.5702
